In [1]:
import pandas as pd
import numpy as np
np.random.seed(10)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [2]:
data = {
    'tra': pd.read_csv('air_visit_data.csv'),
    'as': pd.read_csv('air_store_info.csv'),
    'hs': pd.read_csv('hpg_store_info.csv'),
    'ar': pd.read_csv('air_reserve.csv'),
    'hr': pd.read_csv('hpg_reserve.csv'),
    'id': pd.read_csv('store_id_relation.csv'),
    'sub': pd.read_csv('sample_submission.csv'),
    'hol': pd.read_csv('date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }


In [3]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [4]:
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date   

In [5]:
data['ar'].head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2
4,air_db80363d35f10926,2016-01-01,2016-01-01,5


In [6]:
data['hr'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02,2016-01-02,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03,2016-01-02,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06,2016-01-04,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11,2016-01-11,2,air_db80363d35f10926


In [7]:
data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: 
                                                   (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)

In [8]:
data['ar'].head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2
4,air_db80363d35f10926,2016-01-01,2016-01-01,5


In [9]:
data['hr'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,reserve_datetime_diff
0,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,4,air_db80363d35f10926,0
1,hpg_878cc70b1abc76f7,2016-01-02,2016-01-02,2,air_db80363d35f10926,0
2,hpg_878cc70b1abc76f7,2016-01-03,2016-01-02,6,air_db80363d35f10926,1
3,hpg_878cc70b1abc76f7,2016-01-06,2016-01-04,3,air_db80363d35f10926,2
4,hpg_878cc70b1abc76f7,2016-01-11,2016-01-11,2,air_db80363d35f10926,0


In [10]:
data[df] = data[df].groupby(['air_store_id','visit_datetime'], 
                            as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(
                            columns={'visit_datetime':'visit_date'})

In [11]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [12]:
data['sub']['visit_date'] = data['sub']['id'].map(lambda x: str(x).split('_')[2])
data['sub']['air_store_id'] = data['sub']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['sub']['visit_date'] = pd.to_datetime(data['sub']['visit_date'])
data['sub']['dow'] = data['sub']['visit_date'].dt.dayofweek
data['sub']['year'] = data['sub']['visit_date'].dt.year
data['sub']['month'] = data['sub']['visit_date'].dt.month
data['sub']['visit_date'] = data['sub']['visit_date'].dt.date

In [13]:
data['sub'].head()

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


In [14]:
data['tra'].head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [15]:
unique_stores = data['sub']['air_store_id'].unique()

In [16]:
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 
                                  'dow': [i]*len(unique_stores)}) for i in range(7)], 
                                   axis=0, ignore_index=True).reset_index(drop=True)

In [17]:
stores.head()

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0


In [18]:
data['tra'].head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [19]:
tmp = data['tra'].groupby(['air_store_id','dow'], 
                          as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})

In [20]:
tmp.head()

,air_store_id,dow,min_visitors
0,air_00a91d42b08b08d9,0,1
1,air_00a91d42b08b08d9,1,1
2,air_00a91d42b08b08d9,2,15
3,air_00a91d42b08b08d9,3,15
4,air_00a91d42b08b08d9,4,17


In [21]:
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])

In [23]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
lbl = LabelEncoder()
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['sub'], data['hol'], how='left', on=['visit_date']) 

KeyError: 'air_genre_name'